In [1082]:
import pandas as pd
from etna.datasets import TSDataset
from etna.transforms import LagTransform, DateFlagsTransform
from etna.models import CatBoostPerSegmentModel
from etna.pipeline import Pipeline
from etna.metrics import RMSE


In [1083]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import STL
import seaborn as sns
from scipy import stats
import itertools
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import GridSearchCV, KFold
from catboost import CatBoostRegressor
from etna.models import CatBoostPerSegmentModel
from etna.metrics import RMSE
from etna.transforms import LagTransform, MeanTransform, DateFlagsTransform
from etna.pipeline import Pipeline
from etna.datasets import TSDataset
from sklearn.model_selection import train_test_split
from etna.transforms import DateFlagsTransform, HolidayTransform
from etna.transforms import LagTransform



In [1084]:
import pandas as pd

# Загрузка данных
data_path1 = './hackathon/data1.csv'
data1 = pd.read_csv(data_path1)

data_path2 = './hackathon/data2.csv'
data2 = pd.read_csv(data_path2)

data_path3 = './hackathon/data3.csv'
data3 = pd.read_csv(data_path3)

data_path4 = './hackathon/data4.csv'
data4 = pd.read_csv(data_path4)

data_path5 = './hackathon/data5.csv'
data5 = pd.read_csv(data_path5)

# Преобразуем столбцы и создаем сегменты для каждого набора данных
data1['segment'] = 'segment_1'
data1 = data1.rename(columns={"date": "timestamp", "value": "target"})
data1['timestamp'] = pd.to_datetime(data1['timestamp'])

data2['segment'] = 'segment_2'
data2 = data2.rename(columns={"date": "timestamp", "value": "target"})
data2['timestamp'] = pd.to_datetime(data2['timestamp'])

data3['segment'] = 'segment_3'
data3 = data3.rename(columns={"date": "timestamp", "value": "target"})
data3['timestamp'] = pd.to_datetime(data3['timestamp'])


# Объединяем все данные в один DataFrame
data = pd.concat([data1, data2, data3], ignore_index=True)
data = data.fillna(0)
# Проверяем результат
print(data.head())


   target  timestamp    segment
0   898.0 2022-01-01  segment_1
1  3167.0 2022-01-02  segment_1
2  3380.0 2022-01-03  segment_1
3  3423.0 2022-01-04  segment_1
4  3373.0 2022-01-05  segment_1


In [1085]:
from etna.transforms import MeanTransform
mean_tr = MeanTransform(in_column="target", out_column="mean", window=5)
ts_with_features = TSDataset(data, freq="D")

In [1086]:
import pandas as pd
from etna.datasets import TSDataset

# Предположим, что ts_with_features — это ваш TSDataset
df = ts_with_features.df  # df — это DataFrame внутри TSDataset

# Сортируем данные по временной метке
df = df.sort_values(by="timestamp")

# Получаем последние 30 дней для валидации
valid_df = df[-30:].copy()

# Обучающие данные — это все данные до последних 30 дней
train_df = df[:-30].copy()

# Преобразуем обратно в TSDataset
train_tsd = TSDataset(df=train_df, freq="D")
valid_tsd = TSDataset(df=valid_df, freq="D")

# Проверим размеры и структуру данных
print(f"Размер обучающего набора: {train_df.shape}")
print(f"Размер валидационного набора: {valid_df.shape}")


Размер обучающего набора: (944, 3)
Размер валидационного набора: (30, 3)


In [1087]:
from etna.transforms import (
    LinearTrendTransform,
    DensityOutliersTransform,
    TimeSeriesImputerTransform,
    StandardScalerTransform
)

transforms = [
    DensityOutliersTransform(
        in_column="target",
        window_size=45,
        n_neighbors=25,
        distance_coef=1.9
    ),
    TimeSeriesImputerTransform(
        in_column="target",
        strategy="running_mean"  # линейная интерполяция для пропусков
    ),
    LinearTrendTransform(in_column="target"),  # Тренд
    LagTransform(in_column="target", out_column="lags", lags=[1, 2, 3, 7, 14, 30]),  # Больше лагов
    DateFlagsTransform(out_column="date_flags", 
                       day_number_in_week=True,
                       day_number_in_month=True,
                       week_number_in_month=True,
                       is_weekend=True),
    MeanTransform(in_column="target", out_column="mean", window=5),  # Сглаживание
    StandardScalerTransform(in_column='target', out_column='target')  # Нормализация
]

/home/denis/miniforge3/envs/hackathon_time/lib/python3.10/site-packages/etna/transforms/math/sklearn.py:72: UserWarning: Transformation will be applied inplace, out_column param will be ignored
  warnings.warn("Transformation will be applied inplace, out_column param will be ignored")


In [1088]:
from etna.models import CatBoostPerSegmentModel
from etna.transforms import LagTransform, MeanTransform, DateFlagsTransform
from etna.pipeline import Pipeline
from etna.metrics import RMSE
import numpy as np
import matplotlib.pyplot as plt

# 1. Модель CatBoost с исправленными параметрами
catboost_model = CatBoostPerSegmentModel(
    n_estimators=300,  # Количество деревьев
    learning_rate=0.1,  # Меньшая скорость обучения
    depth=7,  # Глубина деревьев
    random_state=42  # Для воспроизводимости
)

pipe = Pipeline(model=catboost_model, transforms=transforms, 
                                                    horizon=len(valid_tsd.df))

pipe.fit(train_tsd)

forecast_valid = pipe.forecast(train_tsd)

rmse = RMSE()
rmse_value = rmse(y_true=valid_tsd, y_pred=forecast_valid)


/home/denis/miniforge3/envs/hackathon_time/lib/python3.10/site-packages/etna/transforms/decomposition/detrend.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result[self.in_column] = no_trend_timeseries
/home/denis/miniforge3/envs/hackathon_time/lib/python3.10/site-packages/etna/transforms/decomposition/detrend.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result[self.in_column] = no_trend_timeseries
/home/denis/miniforge3/envs/hackathon_time/lib/python3.10/site-packages/etna/transforms/decompos

In [1089]:
rmse_value

{'segment_1': 370.4878071478517,
 'segment_2': 140.43353279401813,
 'segment_3': 85.19416894352366}